In [27]:
r.delete("ranking_jugadores")

1

In [1]:
import tkinter as tk
from tkinter import ttk
import time
import random
import threading
import redis

# Configuración de Redis
r = redis.Redis(
    host='localhost',
    port=6379,
    db=0)

# Lista de nicknames para los jugadores
nicknames = [
    "TheDestroyer", "Lightning", "Shadow", "IronFist", "Phantom", "Speedster", "Vortex", "Blaze", "Crusher", "Dragon",
    "Rogue", "Sniper", "Zephyr", "Ghost", "Titan", "Raider", "Warrior", "Hunter", "Joker", "Samurai",
    "Raven", "Demon", "Ninja", "Wolverine", "Maverick", "Thunder", "Falcon", "Storm", "Eagle", "Predator",
    "Cyborg", "Stryker", "Phoenix", "Reaper", "Hawk", "Viper", "Sphinx", "Assassin", "Knight", "Berserker",
    "Juggernaut", "Sorcerer", "Paladin", "Monk", "Vigilante", "Wraith", "Gladiator", "Barbarian", "Valkyrie", "Sentinel"
]

# Simulación de los jugadores con nicknames
jugadores = [
    {"id": i + 1, "nombre": nicknames[i], "puntos": random.randint(1000, 2000)}
    for i in range(50)
]

# Actualiza los puntajes de los jugadores
def actualizar_puntos(jugador_nombre):
    puntos_actuales = r.zscore("ranking_jugadores", jugador_nombre)
    if puntos_actuales is None:
        puntos_actuales = 0    

    nuevos_puntos = random.randint(100, 6000)
    puntos_finales = max(puntos_actuales + nuevos_puntos, 0)
    timestamp = time.time()

    # Actualizar puntajes en Redis
    r.zadd("ranking_jugadores", {jugador_nombre: puntos_finales})
    r.hset(f"{jugador_nombre}_last_update", "timestamp", timestamp)
    
    # Guardar en el historial
    historial_entrada = f"{time.ctime(timestamp)}: {puntos_finales}"
    r.lpush(f"{jugador_nombre}_historial", historial_entrada)

def mostrar_historial(jugador_nombre):
    historial = r.lrange(f"{jugador_nombre}_historial", 0, -1)  # Obtener todo el historial
    return [entrada.decode('utf-8') for entrada in historial]

# Función para mostrar el top 3 jugadores en la interfaz
def mostrar_top3_ranking():
    ranking = r.zrevrange("ranking_jugadores", 0, 2, withscores=True)  # Top 3 jugadores
    return [(nombre.decode('utf-8'), puntos) for nombre, puntos in ranking]

# Hilo para actualizar puntajes en segundo plano
def actualizar_puntajes_periodicamente():
    contador = 0
    while contador < 10:
        jugador_nombre = random.choice([jugador['nombre'] for jugador in jugadores])  # Selección de nickname
        actualizar_puntos(jugador_nombre)
        contador += 1
        time.sleep(10)

# Hilo para actualizar la interfaz gráfica
def actualizar_interfaz():
    while True:
        top3 = mostrar_top3_ranking()
        for i, (nombre, puntos) in enumerate(top3):
            etiquetas[i].config(text=f"{i+1}. {nombre}: {puntos} puntos")
        time.sleep(1)

# Configuración de la interfaz con Tkinter y ttk
def crear_interfaz():
    ventana = tk.Tk()
    ventana.title("Ranking de Jugadores")
    ventana.geometry("400x350")
    ventana.configure(bg="#1a1a1a")  # Fondo oscuro

    # Usar ttk para widgets modernos
    style = ttk.Style()
    style.theme_use("clam")  # Estilo moderno
    style.configure("TLabel", foreground="#ecf0f1", background="#1a1a1a", font=("Helvetica", 14))

    # Título estilizado
    titulo = ttk.Label(ventana, text="Top 3 Jugadores", font=("Helvetica", 20, "bold"))
    titulo.pack(pady=20)

    # Etiquetas para mostrar el top 3
    global etiquetas
    etiquetas = [ttk.Label(ventana, text="Cargando...", padding=10) for _ in range(3)]
    for etiqueta in etiquetas:
        etiqueta.pack(pady=5)

    # Botón para cerrar la ventana
    boton_cerrar = ttk.Button(ventana, text="Cerrar", command=ventana.destroy)
    boton_cerrar.pack(pady=20)

    # Hilos
    hilo_actualizar_puntajes = threading.Thread(target=actualizar_puntajes_periodicamente, daemon=True)
    hilo_actualizar_puntajes.start()

    hilo_interfaz = threading.Thread(target=actualizar_interfaz, daemon=True)
    hilo_interfaz.start()

    # Iniciar la interfaz gráfica
    ventana.mainloop()

# Ejecutar la interfaz
crear_interfaz()


Exception in thread Thread-6 (actualizar_interfaz):
Traceback (most recent call last):
  File "C:\Users\Usuario 1\anaconda3\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "C:\Users\Usuario 1\anaconda3\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Usuario 1\AppData\Local\Temp\ipykernel_8208\3129587908.py", line 70, in actualizar_interfaz
  File "C:\Users\Usuario 1\anaconda3\Lib\tkinter\__init__.py", line 1722, in configure
    return self._configure('configure', cnf, kw)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Usuario 1\anaconda3\Lib\tkinter\__init__.py", line 1712, in _configure
    self.tk.call(_flatten((self._w, cmd)) + self._options(cnf))
RuntimeError: main thread is not in main loop
